In [1]:
# 이 셀을 실행시켜서 설치가 잘 되었는지 확인해 보세요
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

## Part 1 : 사적인, 탈중앙화된 데이터과학의 기초 도구

In [2]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


### Section 1.1 - Bob의 머신에 텐서를 보내기

Torch 텐서를 이용하는 대신에 **포인터**를 사용하여 텐서를 이용

-  먼저 "가상의" 사람이 소유한 `"가상"` 머신을 만들고 그 사람을 `bob`이라고 합시다.

In [3]:
bob = sy.VirtualWorker(hook, id="bob")

In [4]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

- 그리고 이제 우리의 텐서를 `bob`에게 보냅시다!

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [6]:
x_ptr

(Wrapper)>[PointerTensor | me:37606516160 -> bob:51366039195]

- 짜잔! 이제 `bob`은 두 개(`x`, `y`의 텐서를 가지고 있다.

In [7]:
bob._objects

{51366039195: tensor([1, 2, 3, 4, 5]), 16779588421: tensor([1, 1, 1, 1, 1])}

In [8]:
z = x_ptr + x_ptr

In [9]:
z

(Wrapper)>[PointerTensor | me:95942297935 -> bob:95567755565]

In [10]:
bob._objects

{51366039195: tensor([1, 2, 3, 4, 5]),
 16779588421: tensor([1, 1, 1, 1, 1]),
 95567755565: tensor([ 2,  4,  6,  8, 10])}

우리가 `x.send(bob)`를 실행했을 때 우리는 `x_ptr`이라는 새로운 객체를 반환했습니다. 이것은 텐서의 첫 번째 포인터입니다. 텐서에 대한 포인터는 **실제로 데이터 자체를 보유하지 않습니다**. 대신 다른 머신에 저장된 텐서(데이터 포함)에 대한 **메타 데이터**만 포함합니다. 이 텐서의 목적은 다른 머신에 텐서를 사용하여 함수를 계산하도록 지시하는 직관적인 API를 제공하는 것입니다. 포인터에 포함된 메타 데이터를 살펴보겠습니다.

In [11]:
x_ptr

(Wrapper)>[PointerTensor | me:37606516160 -> bob:51366039195]

**포인터와 관련된 두 가지 주요 속성**

- `x_ptr.location` : `bob` 포인터가 가리키는 위치에 대한 참조
- `x_ptr.id_at_location` : `<random integer>`, 위치에 텐서가 어디에 저장되어있는 지를 구별하는 `id`

**다른 일반적인 속성**
- `x_ptr.id` : <random integer>, 포인터 텐서의 id로 무작위로 할당
- `x_ptr.owner` : "me", 포인터 텐서를 소유한 작업자입니다. 여기서는 "me"라는 로컬 작업자

In [12]:
# bob 포인터가 가리키는 위치에 대한 참조
bob

<VirtualWorker id:bob #objects:3>

In [13]:
# bob 포인터가 가리키는 위치에 대한 참조
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [14]:
x_ptr.location == bob

True

In [15]:
# <random integer>, 위치에 텐서가 어디에 저장되어있는 지를 구별하는 id
x_ptr.id_at_location

51366039195

In [16]:
# "me", 포인터 텐서를 소유한 작업자입니다. 여기서는 "me"라는 로컬 작업자
x_ptr.owner

<VirtualWorker id:me #objects:0>

여기서 포인터를 소유한 로컬 작업자가 `VirtualWorker`인 이유를 궁금해할 수도 있는데, 재미있는 사실은 `Bob`에 대한 `VirtualWorker` 객체가 있는 것처럼 항상 **(기본값으로) 하나를 가지고 있다**는 것입니다. 이 작업자는 우리가 `hook` = `sy.TorchHook()`을 호출할 때 **자동으로 생성**되므로 직접 만들 필요가 없습니다.

In [17]:
# "me", 포인터 텐서를 소유한 작업자입니다. 여기서는 "me"라는 로컬 작업자
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [18]:
me == x_ptr.owner

True

마지막으로 텐서에서 `.send()` 를 호출할 수 있는 것처럼 텐서의 포인터에서 `.get()` 을 호출하여 다시 가져옴

In [19]:
x_ptr

(Wrapper)>[PointerTensor | me:37606516160 -> bob:51366039195]

In [20]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [22]:
y_ptr

(Wrapper)>[PointerTensor | me:5578444465 -> bob:16779588421]

In [23]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [24]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [25]:
bob._objects

{}

- 그리고 위에서 보면 알 수 있듯이... **`bob`은 더 이상 텐서가 없습니다!!!** 우리 머신으로 다시 이동한 것

### section 1.2 - 텐서 포인터 사용하기

In [26]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [28]:
z = x + y

In [29]:
z

(Wrapper)>[PointerTensor | me:90906966523 -> bob:68229227060]

보이지 않는 곳에서 매우 강력한 무언가가 일어났습니다. 
`x`와 `y`가 `local`에서 덧셈을 계산하는 대신, 명령이 직렬화되어 `bob`에게 전송되었으며, `bob`은 계산을 수행하고 텐서 `z`를 만든 다음 `z`에 대한 포인터를 다시 우리에게 돌려주었습니다!

포인터에서 `.get()` 을 호출하면 결과를 컴퓨터로 다시 받습니다!

In [30]:
z.get()

tensor([2, 3, 4, 5, 6])

### Torch 함수들

In [31]:
x

(Wrapper)>[PointerTensor | me:90248025061 -> bob:6438567346]

In [33]:
y

(Wrapper)>[PointerTensor | me:87883083760 -> bob:67070791607]

In [34]:
z = torch.add(x, y)

In [35]:
z.get()

tensor([2, 3, 4, 5, 6])

### 변수들 (backpropagation을 포함)

In [96]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

- `+` 에 대한 항상 `gradient =  1`

In [97]:
z = (x + y).sum()

In [98]:
z.backward()

(Wrapper)>[PointerTensor | me:27097783527 -> bob:57824403967]

In [99]:
x = x.get()

In [100]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [101]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [102]:
y = y.get()

In [103]:
y

tensor([1., 1., 1., 1., 1.], requires_grad=True)

In [104]:
y.grad

tensor([1., 1., 1., 1., 1.])

`*` 에 대한 `gradient`는 서로 상반된 `element` 확인

In [110]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)
z = (x * y).sum()
z.backward()
x, y = x.get(), y.get()
print("x's gradient : ",x.grad)
print("---------------------------------------------")
print("y's gradient : ",y.grad)

x's gradient :  tensor([1., 1., 1., 1., 1.])
---------------------------------------------
y's gradient :  tensor([1., 2., 3., 4., 5.])
